In [79]:
%load_ext autoreload
%autoreload 2

from NewsContent import *
from UserContent import *
from preprocessing import *
from PEGenerator import *
import PEGenerator
from models import *
from utils import *
from Encoders import *

import os
import numpy as np
import json
import random

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [80]:
data_root_path = "../data/Challenge/"
embedding_path = "../../"
KG_root_path = None
popularity_path = "../data/Challenge/popularity"
config = {'title_length':30,
              'body_length':100,
              'max_clicked_news':50,
              'npratio':1,
              'news_encoder_name':"CNN",
              'user_encoder_name':"Att",
             'attrs':['title','body','vert'],
             'word_filter':0,
             'data_root_path':data_root_path,
             'embedding_path':embedding_path,
             'KG_root_path':KG_root_path,
            'popularity_path':popularity_path,
             'max_entity_num':5}
model_config = {
        'news_encoder':0,
        'popularity_user_modeling':True,
        'rel':True,
        'ctr':True,
        'content':True,
        'rece_emb':True,
        'activity':True

    }

In [81]:
News = NewsContent(config)
TrainUsers = UserContent(News.news_index,config,'train.tsv',2)
ValidUsers = UserContent(News.news_index,config,'val.tsv',1)
TestUsers = UserContent(News.news_index,config,'test.tsv',2)

19779
2472
2473


In [82]:
train_sess,train_buckets, train_user_id, train_label = get_train_input(TrainUsers.session,News.news_index,config)
test_impressions, test_userids = get_test_input(TestUsers.session,News.news_index)
val_impressions, val_userids = get_test_input(ValidUsers.session,News.news_index)

19779
19916
2473
2472


In [83]:
print(News.news_stat_imp[train_sess[0, 0],train_buckets[7]])

0.0


In [84]:
print(News.news_publish_bucket[train_sess[0,0]])

548


In [85]:
title_word_embedding_matrix, have_word = load_matrix(embedding_path,News.word_dict)

In [93]:
class EbnerdTrainData(Dataset):
    def __init__(self, News, User, session, user_id, buckets, labels):
        self.News = News
        self.User = User
        self.session = session
        self.buckets = buckets
        self.user_id = user_id
        self.labels = labels
        self.length = len(self.session)
        # ?
        # self.batch_size = batch_size
        
    def __len__(self):
        return self.length
    
    def __getitem__(self, idx):
        candidates = torch.IntTensor(self.session[idx])
        candidates_data = torch.LongTensor(self.News.fetch_news(candidates))
        candidates_ctr = torch.FloatTensor([self.News.get_ctr(x, self.buckets[idx]) for x in candidates])
        candidates_rece_emb_index = torch.IntTensor([self.News.news_publish_bucket2[x] for x in candidates])
        user_activity_input = 1
        clicked_input = torch.IntTensor(self.User.click[self.user_id[idx]])
        clicked_input_data = torch.LongTensor(self.News.fetch_news(clicked_input))
        # print(clicked_input)
        clicked_ctr = torch.FloatTensor([self.News.get_ctr(self.User.click[self.user_id[idx]][i], self.User.click_bucket[self.user_id[idx]][i]) for i in range(len(clicked_input))])
        # print(clicked_ctr)
        input = [candidates_data,candidates_ctr,candidates_rece_emb_index,user_activity_input,clicked_input_data,clicked_ctr]
        # for x in range(len(input)):
        #     input[x] = torch.FloatTensor(input[x])
        labels = torch.LongTensor(self.labels[idx])
        return input, labels



In [87]:
class EbnerdTestData(Dataset):
    def __init__(self, News, User, user_id, impressions):
        self.News = News
        self.User = User
        self.impressions = impressions
        self.user_id = user_id
        
        self.length = len(self.user_id)
        
    def __len__(self):
        return self.length
    
    def __getitem__(self, idx):
        candidates = self.impressions[idx]['docs']
        candidates_ctr = [self.News.get_ctr(x, self.impressions[idx]['tsp']) for x in candidates]
        candidates_rece_emb_index = [self.News.news_publish_bucket2[x] for x in candidates]
        user_activity_input = 1
        clicked_input = self.User.click[self.user_id[idx]]
        # print(clicked_input)
        clicked_ctr = [self.News.get_ctr(self.User.click[self.user_id[idx]][i], self.User.click_bucket[self.user_id[idx]][i]) for i in range(len(clicked_input))]
        # print(clicked_ctr)
        input = [candidates,candidates_ctr,candidates_rece_emb_index,user_activity_input,clicked_input,clicked_ctr]
        for x in range(len(input)):
            input[x] = torch.FloatTensor(input[x])
        labels = self.impressions[idx]['labels']

        return input, labels


In [100]:
# train_generator = TrainGenerator(News,TrainUsers,train_sess,train_user_id,train_buckets,train_label,32)
train_loader = DataLoader(EbnerdTrainData(News, TrainUsers, train_sess, train_user_id, train_buckets, train_label), 1)
# # test_user_generator = UserGenerator(News,TestUsers,32)
# test_loader = DataLoader(EbnerdTestData(News, TestUsers, test_userids, test_impressions), 32)
# # val_user_generator = UserGenerator(News,ValidUsers,32)

# val_loader = DataLoader(EbnerdTestData(News, ValidUsers, val_userids, val_impressions), 8)
# # news_generator = NewsGenerator(News,32)



In [101]:
News.category_dict

{'sex_og_samliv': 0,
 'podcast': 1,
 'nyheder': 2,
 'bibliotek': 3,
 'penge': 4,
 'services': 5,
 'krimi': 6,
 'biler': 7,
 'om_ekstra_bladet': 8,
 'nationen': 9,
 'haandvaerkeren': 10,
 'musik': 11,
 'play': 12,
 'sport': 13,
 'horoskoper': 14,
 'underholdning': 15,
 'opinionen': 16,
 'side9': 17,
 'forbrug': 18,
 'incoming': 19,
 'auto': 20,
 'ferie': 21,
 'dagsorden': 22,
 'vin': 23,
 'plus': 24}

In [102]:
for x, y in train_loader:
    print(x[0][0])
    break

{'title': array([[ 6559,  7914,   213,   138, 12011,   275,   374,    64,     6,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0],
       [ 1507,  1862,   153,   119,  2497,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0]]), 'vert': array([[ 2],
       [13]]), 'subvert': None, 'body': array([[   113,     67,    697,    213,     64,     54,   4369,   7914,
        120597,   6360,     13, 120598, 120599,  23690,     11,   5208,
          2314,   3548,     21,     76,   5525,    179, 120600,  21950,
            36,    122,     30,   1628,    337,  18535,    225, 120601,
            21,    300,    479,     98,    595,  17715,      8,   1885,
           225,    113, 120602,     11,      9,   8381, 120603,     36,
         

In [121]:
from torch.optim import Adam

model,user_encoder,news_encoder,bias_news_encoder,bias_content_scorer,scaler,time_embedding_layer,activity_gater = \
create_pe_model(config, model_config, News, title_word_embedding_matrix, entity_embedding_matrix=None)


50 400


In [124]:
# Step 2: Create your Adam optimizer
optimizer = Adam(model.parameters(), lr=0.0001)

loss_fn = nn.CrossEntropyLoss()

# Step 3: Iterate over the data for the number of epochs
for epoch in range(1):

    # Step 4: Iterate over each batch of data and compute the scores using the forward pass of the network
    model.train()
    for x, y in train_loader:
        optimizer.zero_grad()
        out = model(x)
    
        # Step 5: Compute the lambda gradient values for the pairwise loss (spedup) with the compute_lambda_i method on the scores and the output labels
        loss = loss_fn(out, y)

        # Step 6: Bacward from the scores with the use of the lambda gradient values
        if loss is not None:
            torch.autograd.backward(out, loss)
            
            # Step 7: Update the weights using the optimizer
            optimizer.step()

    # Step 8: At the end of the epoch, evaluate the model on the data using the evaluate_model function (both train and val)
    model.eval()
#     val_rankings = news_ranking(model_config,ctr_weight,val_predicted_activity_gates,val_user_scoring,news_scoring,
#                                bias_candidate_score,news_bias_vecs,time_embedding_matrix,bias_content_scorer,
#                                News,val_impressions)
#     train_metrics = evaluate_model(data, model.forward, 'train')
#     val_metrics = evaluate_model(data, model.forward, 'validation')
    
        
#     print(f"Epoch {epoch+1}/{params.epochs} - Train Metrics: {train_metrics}, Val Metrics: {val_metrics}")

#     # Step 9: Append the metrics to train_metrics_epoch and val_metrics_epoch
#     train_metrics_epoch.append(train_metrics)
#     val_metrics_epoch.append(val_metrics)
    
# return {
#     "metrics_train": train_metrics_epoch,
#     "metrics_val": val_metrics_epoch
# }

a
torch.Size([2, 30])
torch.Size([2, 1, 30])
30 400
torch.Size([2, 1, 30])
torch.Size([2, 1, 30, 300])


RuntimeError: Expected 2D (unbatched) or 3D (batched) input to conv1d, but got input of size: [2, 1, 30, 300]

In [ ]:
# model.compile(loss=['categorical_crossentropy'],
#                   optimizer=Adam(lr=0.0001), 
#                   metrics=['acc'])

In [ ]:
for i in range(10):

    model,user_encoder,news_encoder,bias_news_encoder,bias_content_scorer,scaler,time_embedding_layer,activity_gater = create_pe_model(config,model_config,News,title_word_embedding_matrix,News.entity_embedding)
    model.fit_generator(train_generator,epochs=2)
    news_scoring = news_encoder.predict_generator(news_generator,verbose=True)
    user_scoring = user_encoder.predict_generator(test_user_generator,verbose=True)
    val_user_scoring = user_encoder.predict_generator(val_user_generator,verbose=True)


    news_bias_vecs = bias_news_encoder.predict_generator(news_generator,verbose=True)

    if model_config['content'] and not model_config['rece_emb']:
        bias_candidate_score = bias_content_scorer.predict(news_bias_vecs,batch_size=32,verbose=True)
        bias_candidate_score = bias_candidate_score[:,0]
    else:
        bias_candidate_score = 0

    ctr_weight = scaler.get_weights()[0][0,0]
    time_embedding_matrix = time_embedding_layer.get_weights()[0]
    
    predicted_activity_gates = activity_gater.predict(user_scoring,verbose=True)
    predicted_activity_gates = predicted_activity_gates[:,0]
    
    val_predicted_activity_gates = activity_gater.predict(val_user_scoring,verbose=True)
    val_predicted_activity_gates = val_predicted_activity_gates[:,0]
    
    rankings = news_ranking(model_config,ctr_weight,predicted_activity_gates,user_scoring,news_scoring,
                                bias_candidate_score,news_bias_vecs,time_embedding_matrix,bias_content_scorer,
                                News,test_impressions)
    
    val_rankings = news_ranking(model_config,ctr_weight,val_predicted_activity_gates,val_user_scoring,news_scoring,
                               bias_candidate_score,news_bias_vecs,time_embedding_matrix,bias_content_scorer,
                               News,val_impressions)
    
    performance = evaluate_performance(rankings,test_impressions)
    val_performance = evaluate_performance(val_rankings,val_impressions)

    cold = []
    for TOP_COLD_NUM in [0,1,3,5,]:
        g = evaluate_cold_users(rankings,test_impressions,TestUsers.click,TOP_COLD_NUM)
        cold.append(g)
    diversity = []
    for TOP_DIVERSITY_NUM in range(1,11):
        div_top = evaluate_diversity_topic_all(TOP_DIVERSITY_NUM,rankings,test_impressions,News,TestUsers)
        div_ilxd = evaluate_density_ILxD(TOP_DIVERSITY_NUM,rankings,test_impressions,news_scoring)
        diversity.append([div_top,div_ilxd])
